In [5]:
#wersja z okrągłym (bez elipsy) ruchem ramon

In [6]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import interpolate
from scipy.integrate import ode
import matplotlib.colors as colors 
import matplotlib.cbook as cbook
from matplotlib import cm
import imageio
from mpl_toolkits import axes_grid1


In [7]:
a=5
b= 100
xx = np.linspace(-a, a, b)
yy = np.linspace(-a, a, b)
mX, mY = np.meshgrid(xx,yy)

In [8]:
def stokeslet(f,r0,mX,mY):
    Id=np.array([[1,0],[0,1]])
    r=np.array([mX-r0[0],mY-r0[1]])

    Idf=np.dot(Id,f) 
    
    rTf=(r*f[:,np.newaxis,np.newaxis]).sum(axis=0)
    rrTf=(r*rTf[np.newaxis,])
    modr=(r[0]**2+r[1]**2)**.5
    
    u,v=Idf[:,np.newaxis,np.newaxis]/modr[np.newaxis]+rrTf/modr**3.
    return [u,v]

def B_dir(t,p,fx,fz):
    ex = fx(p[0],p[1])
    ez = fz(p[0],p[1])
    n = (ex**2.0+ez**2.0)**0.5
    return [ex/n, ez/n]

def add_colorbar(im, aspect=20, pad_fraction=0.5, **kwargs):
    """Add a vertical color bar to an image plot."""
    divider = axes_grid1.make_axes_locatable(im.axes)
    width = axes_grid1.axes_size.AxesY(im.axes, aspect=1./aspect)
    pad = axes_grid1.axes_size.Fraction(pad_fraction, width)
    current_ax = plt.gca()
    cax = divider.append_axes("right", size=width, pad=pad)
    plt.sca(current_ax)
    return im.axes.figure.colorbar(im, cax=cax, **kwargs)

In [9]:
R=0.001
dt=0.8*R
r1=np.array([2/np.sqrt(3),1])   # f is direction of the force
f=np.array([0,1])                         # r is position of the force red arrow - touchdown point
r2=np.array([-2/np.sqrt(3),1])
r3=np.array([0,0])
x0, x1= -0.99*a, 0.99*a
y0, y1= -0.99*a, 0.99*a

In [10]:

t = 10
y = np.linspace(np.sin(np.arctan(2/np.sqrt(3))), np.sin(0) , t)

amy = np.zeros(2*t) 
for i in range(0, t):
    
    amy[i] = y[i]
    x = np.flip(y)
    amy[i+10] = x[i]

amx = np.zeros(t*2)
t=5
x = np.linspace(np.sin(np.arctan(2/np.sqrt(3))), np.cos(0) , t)
k = x[1]
for i in range(0, t):
    amx[i] = x[i]
    y = np.flip(x)
    amx[i+t] = y[i]
    amx[i+t*2] = k
    amx[i+t*2+t] = k

amcordinates= np.vstack([amx,amy]).T
names=[amcordinates]




In [11]:


for j in amcordinates:
    fig=plt.figure(figsize=(6,6),facecolor="w")
    ax = plt.axes()
    u1,v1=stokeslet((-.5)*f,np.array([r1[0]*j[0], r1[1]*j[1]]),mX,mY)
    u2,v2=stokeslet((-.5)*f,np.array([r2[0]*j[0], r2[1]*j[1]]),mX,mY)
    u3,v3=stokeslet(f,r3,mX,mY)
    u=u1+u2+u3
    v=v1+v2+v3
    
    # set the starting point of the magnetic field line
    xstart = np.linspace(-0.99*a, 0.99*a, 14)
    additional = np.linspace(-0.99*a, 0.99*a, 8)
    
    ystart = np.zeros(22)
    for i in additional:
        xstart = np.append(xstart, -i)

    for i in range(0, 22):
        if i<14:
            ystart[i]=0.99*a
        if i>=14:
            ystart[i]=-0.99*a


    places=np.vstack([xstart,ystart]).T

    fbx = interpolate.interp2d(xx,yy,u)
    fbz = interpolate.interp2d(xx,yy,v)

    r=ode(B_dir)
    r.set_integrator('vode')
    r.set_f_params(fbx,fbz)

    xs,ys = [],[]
    for p in places:
        x=[p[0]] 
        y=[p[1]]
        r.set_initial_value([p[0], p[1]], 0)
        while r.successful():
            r.integrate(r.t+dt)
            x.append(r.y[0])
            y.append(r.y[1])
            hit_electrode=False
            if (not (x0<r.y[0] and r.y[0]<x1)) or (not (y0<r.y[1] and r.y[1]<y1)):
                break
        xs.append(x)
        ys.append(y)
        
        
    '''p, result = [], []
        
    for x in xs:
        p.append(len(x))

    [result.append(x) for x in p if x not in result]

    result.sort()
    
        
    k = result[-1]
    
    
    l, m, k, M = result[0], result[1], result[-1], result[-2]
    h = int(l/200)
    p = int(m/200)
    g = int(M/200)
    
    alpha = 0'''
    #strzałki są w tym samym miejscu proporcjonalnie, ale kazda kolejna linia jest dłuzsza więc są relatywnie wyzej 
    
    #for x,y in zip(xs,ys):
            #alpha = alpha + 1
     #   ax.plot(x,y, color="k" , zorder=10)
    '''if len(x) == M:
                ax.arrow(x[100*g], y[100*g], (x[100*g-1*alpha]-x[100*g-2*alpha]), (y[100*g+5*alpha]-y[100*g+4*alpha]), length_includes_head=True, head_width=.15, color="k", zorder=5)
                ax.arrow(x[190*p], y[190*p], (x[190*p-1*alpha]-x[190*p-2*alpha]), (y[190*p+5*alpha]-y[190*p+4*alpha]), length_includes_head=True, head_width=.15, color="k", zorder=5)
            if len(x)<= k and len(x)>=m:
                ax.arrow(x[190*p], y[190*p], (x[190*p-1*alpha]-x[190*p-2*alpha]), (y[190*p+5*alpha]-y[190*p+4*alpha]), length_includes_head=True, head_width=.15, color="k", zorder=5)
        '''
        
    Z = np.sqrt(v**2+u**2)
    ax.pcolormesh(mX, mY, Z,
                norm=colors.LogNorm(vmin= 10**(-4), vmax=10**2),
                      snap=True,
               cmap=plt.cm.inferno, rasterized=True, 
               shading='gouraud', zorder=0)
    
    #for x,y in zip(xs,ys):
     #   ax.plot(x,y, color="k" , zorder=10)

    '''ax.arrow(x=0, y=0, dx=0, dy=1, head_width = 0.2,
          width = 0.05,
          color ='yellow', zorder=10) 
    ax.arrow(x=r1[0]*j[0], y=r1[1]*j[1], dx=0, dy=-.5, head_width = 0.2,
          width = 0.05,
          color ='yellow', zorder=10) 
    ax.arrow(x=r2[0]*j[0], y=r2[1]*j[1], dx=0, dy=-.5, head_width = 0.2,
          width = 0.05,
          color ='yellow', zorder=10) '''
    plt.streamplot(mX, mY, u, v, 
               broken_streamlines=False, density=.5, color='k')
    
    image = ax.pcolormesh(mX, mY, Z,
                norm=colors.LogNorm(vmin= 10**(-4), vmax=10**4),    
                #norm=colors.LogNorm(vmin=Z.min(), vmax=Z.max()), 
                      snap=True,
               cmap=plt.cm.inferno, rasterized=True, 
               shading='gouraud', zorder=0)

    add_colorbar(image)
    plt.savefig('plot' + str(j) + '.png',
                bbox_inches='tight', pad_inches=0, dpi=400)
    
    plt.close(fig='all')


/var/folders/95/7_vspww56v5bfch3bd_lv06c0000gn/T/ipykernel_10736/206731167.py:27: DeprecationWarning: `interp2d` is deprecated!
`interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.12.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In new code, for regular grids use `RegularGridInterpolator` instead.
For scattered data, prefer `LinearNDInterpolator` or
`CloughTocher2DInterpolator`.

For more details see
`https://gist.github.com/ev-br/8544371b40f414b7eaf3fe6217209bff`

  fbx = interpolate.interp2d(xx,yy,u)
/var/folders/95/7_vspww56v5bfch3bd_lv06c0000gn/T/ipykernel_10736/206731167.py:28: DeprecationWarning: `interp2d` is deprecated!
`interp2d` is deprecated in SciPy 1.10 and will be removed in SciPy 1.12.0.

For legacy code, nearly bug-for-bug compatible replacements are
`RectBivariateSpline` on regular grids, and `bisplrep`/`bisplev` for
scattered 2D data.

In ne

In [12]:
print(amcordinates)


[[0.75592895 0.75592895]
 [0.81694671 0.67193684]
 [0.87796447 0.58794474]
 [0.93898224 0.50395263]
 [1.         0.41996053]
 [1.         0.33596842]
 [0.93898224 0.25197632]
 [0.87796447 0.16798421]
 [0.81694671 0.08399211]
 [0.75592895 0.        ]
 [0.81694671 0.        ]
 [0.81694671 0.08399211]
 [0.81694671 0.16798421]
 [0.81694671 0.25197632]
 [0.81694671 0.33596842]
 [0.81694671 0.41996053]
 [0.81694671 0.50395263]
 [0.81694671 0.58794474]
 [0.81694671 0.67193684]
 [0.81694671 0.75592895]]


In [13]:
import os
import imageio.v2 as imageio

In [14]:
names= []
for j in amcordinates:
    names.append('plot'+str(j)+'.png')


In [16]:
with imageio.get_writer('zamknięty_tor+strzalki.mp4', mode='I', format="FFMPEG") as writer:
    for filename in names:
        image = imageio.imread(filename)
        writer.append_data(image)

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (2148, 1962) to (2160, 1968) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


In [17]:
for filename in set(names):
     os.remove(filename)